In [4]:
%matplotlib inline

# Import custom scripts

In [5]:
path_scripts = '/Users/simjoo/Projects/10X_Skin/scripts/'

In [6]:
import sys
sys.path.append(path_scripts)

In [7]:
from Skin10X_misc_scripts_v1_4 import *
from Skin10X_plot_v1_0 import *
from Skin10X_affinity_propagation_v1_4 import *
from Skin10X_mann_whitney_u_v1_3 import *

/Users/simjoo/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
import pickle
import pygraphviz as pgv

# Initialize engines

In [9]:
from ipyparallel import Client

In [10]:
c = Client(profile='default')

In [11]:
dview = c[:]

In [12]:
dview.execute('import sys')
dview.execute('sys.path.append("/Users/simjoo/Projects/10X_Skin/scripts/")')
dview.execute('from Skin10X_affinity_propagation_v1_4 import *')
dview.execute('Skin10X_mann_whitney_u_v1_4 import *')

<AsyncResult: execute>

# Import Data

In [13]:
exp_id = '201806201840'
path_input = '/Users/simjoo/Projects/10X_Skin/data_input/v1.5/'
path_output = '/Users/simjoo/Projects/10X_Skin/data_output/v1.5'
path_figures = '/Users/simjoo/Projects/10X_Skin/figures/v1.5'

In [14]:
seq = loadData_v1(path_output, exp_id, 'seq', 'DataFrame')

In [15]:
seqN = loadData_v1(path_output, exp_id, 'seqN', 'DataFrame')

In [16]:
seqN_log2 = np.log2(seqN + 1)

In [17]:
cl_c_1st = loadData_v1(path_output, exp_id, 'cl_c_1st', 'Series')

In [18]:
cl_c_1st_sel = cl_c_1st[cl_c_1st!='JUNK']

In [19]:
cl_c_2nd = loadData_v1(path_output, exp_id, 'cl_c_2nd', 'Series')

In [20]:
cl_c_2nd_sel = cl_c_2nd[cl_c_2nd!='JUNK']

In [21]:
MWU_av_2nd_lig = loadData_v1(path_output, exp_id, 'MWU_av_2nd_lig','DataFrame')
MWU_av_2nd_rec = loadData_v1(path_output, exp_id, 'MWU_av_2nd_rec','DataFrame')

In [22]:
PairsLigRec_mm = loadData_v1(path_input, exp_id, 'PairsLigRec_mm', 'DataFrame')
PairsLigRec_mm_S = pd.Series(PairsLigRec_mm['Receptor.ApprovedSymbol_mm'].values, index = PairsLigRec_mm['Ligand.ApprovedSymbol_mm'].values)

In [23]:
cells_stage = pd.Series([c[-2:] for c in seq.columns], index = seq.columns)

In [24]:
cc_score_v2 = loadData_v1(path_output, exp_id, 'cc_score_v2','Series')

In [25]:
TF_mm9 = open('%s/TF_mm9.txt' % path_input, 'r').read().split()

In [26]:
bc = loadData_v1(path_input, exp_id, 'assigned_barcodes', 'Series')
cells_replicate = pd.Series(index = seq.columns)
for ix in cells_replicate.index:
    if type(bc['%s-1' % ix[:-3]]) != str:
        cells_replicate[ix] = '%s unassigned' % ix.split('-')[-1]
    elif bc['%s-1' % ix[:-3]] == 'unassigned':
        cells_replicate[ix] = '%s unassigned' % ix.split('-')[-1]
    else:
        cells_replicate[ix] = bc['%s-1' % ix[:-3]]

In [27]:
path = '%s/%s_cmap_1st.pkl' % (path_output, exp_id)
f = open(path, 'rb')
cmap_1st = pickle.load(f)

In [28]:
path = '%s/%s_cmap_2nd.pkl' % (path_output, exp_id)
f = open(path, 'rb')
cmap_2nd = pickle.load(f)

In [29]:
cmap_time = {'5w':'#fb9a99','9w':'#b2df8a'}

In [30]:
markers_time = {'5w':'H','9w':'o'}

In [31]:
def quantify_LigRec(gene_dict_Lig, gene_dict_Rec, pairs):
    
    output_df = pd.DataFrame(index = gene_dict_Lig.keys(), columns = gene_dict_Rec.keys())
    output_dict = {}
    
    output_quant_df = pd.DataFrame(index = gene_dict_Lig.keys(), columns = gene_dict_Rec.keys())
    output_quant_dict = {}
    
    for kLig in gene_dict_Lig.keys():
        lig_exp = gene_dict_Lig[kLig]
        
        for kRec in gene_dict_Rec.keys():
            rec_exp = gene_dict_Rec[kRec]
                
            rl = []
            
            for l in lig_exp:
                for r in rec_exp:
                    if np.any(pairs.loc[l] == r):
                        rl.append((l,r))
            
            output_df.loc[kLig,kRec] = rl
            output_dict['%s - %s' % (kLig,kRec)] = rl
            
            output_quant_df.loc[kLig,kRec] = len(rl)
            output_quant_dict['%s - %s' % (kLig,kRec)] = len(rl)
            
    return output_df, output_dict, output_quant_df, output_quant_dict

In [32]:
def sim_lig_rec(dict_rec, dict_lig, pairs, repeats = 10000):
    
    output = {}
    
    for kLig in dict_lig.keys():
        nLig = len(dict_lig[kLig])
        
        for kRec in dict_rec.keys():
            nRec = len(dict_lig[kRec])
            
            output_tmp = []

            for r in range(repeats):
                gLig = np.random.choice(list(set(pairs.index)), size = nLig, replace = False)
                gRec = np.random.choice(list(set(pairs.values)), size = nLig, replace = False)

                output_tmp.append(len([g for g in gRec if g in list(pairs.ix[gLig].values)]))

            output['%s - %s' % (kLig,kRec)] = output_tmp
            
    return output

In [33]:
def sim_lig_rec_sel(dict_rec, dict_lig, pairs_sel, pairs, repeats = 10000):
    
    output = {}
    
    for p in pairs_sel:
        kLig = p[0]
        nLig = len(dict_lig[kLig])
        
        kRec = p[1]
        nRec = len(dict_lig[kRec])
            
        output_tmp = []

        for r in range(repeats):
            gLig = np.random.choice(list(set(pairs.index)), size = nLig, replace = False)
            gRec = np.random.choice(list(set(pairs.values)), size = nLig, replace = False)

            output_tmp.append(len([g for g in gRec if g in list(pairs.ix[gLig].values)]))

        output['%s - %s' % (kLig,kRec)] = output_tmp
            
    return output

In [34]:
def sim_lig_reg_get_p(obs, sim):
    
    output_df = pd.DataFrame()
    
    for k in sim.keys():
        output_df.loc[k.split(' - ')[0], k.split(' - ')[1]] = np.sum(np.array(sim[k])>=obs[k]) / len(sim[k])
        
    return output_df

In [35]:
def create_MultiGraph_pgv(mat, prog = 'neato'):
    
    G=pgv.AGraph(strict = False)
    
    G.add_nodes_from(mat.index)
    
    for n1, n2 in itertools.product(mat.index):
        G.add_edge([])
    
    return G

# 1st level clusters

In [36]:
MWU_lig = MWU_av_2nd_lig
MWU_rec = MWU_av_2nd_rec

## Test for enrichment to define groups for time points

In [40]:
def stage_get_pval_binomial_distr(n, p, N):
    
    """
    n: number of trials == number of cells in a population
    p: success probability == percentage of anagen cells in whole dataset
    N: number of successes == number of anagen cells in population
    -----
    returns
    pval telogen enrichment: probability to observe N or less anagen cells at random
    pval anagen enrichment: probability to observe n-N or less telogen cells at random
    """
    
    return scipy.stats.binom.cdf(N, n, p), scipy.stats.binom.cdf(n-N, n, 1-p)

In [41]:
perc_anagen_all = 0.29973024526812425 #see Notebook 1

In [42]:
stage_quant_1st = pd.DataFrame(index = return_unique(cl_c_1st_sel), columns = ['count anagen',
                                                                           '% anagen',
                                                                           'count telogen',
                                                                           '% telogen',
                                                                           'proba - 5w',
                                                                           'proba - 9w'])

for gr in return_unique(cl_c_1st_sel):
    c_sel = cl_c_1st_sel[cl_c_1st_sel==gr].index
    count_tmp = Counter(cells_stage[c_sel])
    
    stage_quant_1st.loc[gr,'count anagen'] = count_tmp['5w']
    stage_quant_1st.loc[gr,'% anagen'] = stage_quant_1st.loc[gr,'count anagen'] / len(c_sel)
    stage_quant_1st.loc[gr,'count telogen'] = count_tmp['9w']
    stage_quant_1st.loc[gr,'% telogen'] = stage_quant_1st.loc[gr,'count telogen'] / len(c_sel)
    stage_quant_1st.loc[gr,['proba - 9w','proba - 5w']] = stage_get_pval_binomial_distr(len(c_sel), perc_anagen_all, stage_quant_1st.loc[gr,'count anagen'])

In [43]:
stage_quant_1st

,count anagen,% anagen,count telogen,% telogen,proba - 5w,proba - 9w
EPI,851,0.302739,1960,0.697261,0.37067,0.644744
ANA,1959,0.992401,15,0.00759878,0,1
FIB,90,0.233766,295,0.766234,0.998378,0.0023517
IMM,63,0.3,147,0.7,0.522612,0.537371
VASC,31,0.187879,134,0.812121,0.99961,0.000756808
MEL,127,0.881944,17,0.118056,4.50718e-48,1
MISC,28,0.358974,50,0.641026,0.154251,0.895697


In [44]:
thr = 0.001 / len(set(cl_c_1st_sel))

In [45]:
thr

0.00014285714285714287

In [46]:
gr_1st_5w_un = stage_quant_1st['proba - 5w'][stage_quant_1st['proba - 5w']<thr].index

In [47]:
gr_1st_9w_un = stage_quant_1st['proba - 9w'][stage_quant_1st['proba - 9w']<thr].index

In [48]:
gr_1st_5w = [gr for gr in set(cl_c_1st_sel) if gr not in gr_1st_9w_un]

In [49]:
gr_1st_5w

['ANA', 'MISC', 'IMM', 'MEL', 'FIB', 'VASC', 'EPI']

In [50]:
gr_1st_9w = [gr for gr in set(cl_c_1st_sel) if gr not in gr_1st_5w_un]

In [51]:
gr_1st_9w

['MISC', 'IMM', 'FIB', 'VASC', 'EPI']

## Get receptor and ligands (combined from 2nd level subclusters)

In [46]:
thr = -np.log10(0.001)

In [47]:
lig_1st = {'EPI':[],'ANA':[],'FIB':[],'IMM':[],'VASC':[],'MEL':[],'MISC':[]}

for gr in return_unique(cl_c_2nd_sel):
    lig_tmp = list(MWU_lig[gr][MWU_lig[gr]>thr].index)
    lig_1st[gr.split(' - ')[0]] += lig_tmp
        
for k in lig_1st.keys():
    lig_1st[k] = list(set(lig_1st[k]))

In [48]:
for k in lig_1st.keys():
    print(k, len(lig_1st[k]))

EPI 109
ANA 132
FIB 162
IMM 79
VASC 93
MEL 66
MISC 14


In [49]:
lig_1st_5w = {k:lig_1st[k] for k in gr_1st_5w}
lig_1st_9w = {k:lig_1st[k] for k in gr_1st_9w}

In [50]:
rec_1st = {'EPI':[],'ANA':[],'FIB':[],'IMM':[],'VASC':[],'MEL':[],'MISC':[]}

for gr in return_unique(cl_c_2nd_sel):
    rec_tmp = list(MWU_rec[gr][MWU_rec[gr]>thr].index)
    rec_1st[gr.split(' - ')[0]] += rec_tmp
        
for k in rec_1st.keys():
    rec_1st[k] = list(set(rec_1st[k]))

In [51]:
for k in rec_1st.keys():
    print(k, len(rec_1st[k]))

EPI 106
ANA 137
FIB 97
IMM 81
VASC 94
MEL 70
MISC 12


In [52]:
rec_1st_5w = {k:rec_1st[k] for k in gr_1st_5w}
rec_1st_9w = {k:rec_1st[k] for k in gr_1st_9w}

In [53]:
for k in rec_1st.keys():
    print(k, np.sqrt((len(rec_1st[k]) + len(lig_1st[k])) / np.pi))

EPI 8.272643200907131
ANA 9.25339718068125
FIB 9.079772052293041
IMM 7.136496464611085
VASC 7.715176518808164
MEL 6.579524642479541
MISC 2.876813695875796


In [54]:
for k in rec_1st.keys():
    print(k, len(rec_1st[k]) + len(lig_1st[k]))

EPI 215
ANA 269
FIB 259
IMM 160
VASC 187
MEL 136
MISC 26


## Get receptor-ligand pairs

In [55]:
ligrec_1st_5w_df, ligrec_1st_5w_dict, quant_ligrec_1st_5w_df, quant_ligrec_1st_5w_dict = quantify_LigRec(lig_1st_5w, 
                                                                                                         rec_1st_5w,
                                                                                                         PairsLigRec_mm_S)

In [56]:
quant_ligrec_1st_5w_df

,IMM,MISC,MEL,VASC,ANA,EPI,FIB
IMM,79,6,55,66,82,52,63
MISC,8,4,15,17,22,16,23
MEL,19,10,59,70,109,56,68
VASC,50,9,76,105,133,77,120
ANA,48,16,83,127,208,133,122
EPI,46,13,72,101,161,98,98
FIB,82,18,138,169,234,139,189


In [57]:
ligrec_1st_9w_df, ligrec_1st_9w_dict, quant_ligrec_1st_9w_df, quant_ligrec_1st_9w_dict = quantify_LigRec(lig_1st_9w, 
                                                                                                         rec_1st_9w,
                                                                                                         PairsLigRec_mm_S)

In [58]:
quant_ligrec_1st_9w_df

,IMM,MISC,VASC,EPI,FIB
IMM,79,6,66,52,63
MISC,8,4,17,16,23
VASC,50,9,105,77,120
EPI,46,13,101,98,98
FIB,82,18,169,139,189


### Simulate background distribution

In [636]:
sim_lig_rec_1st_5w = sim_lig_rec(rec_1st_5w, lig_1st_5w, PairsLigRec_mm_S, repeats = 10000)

/Users/simjoo/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [686]:
quant_sim_lig_rec_1st_5w = sim_lig_reg_get_p(quant_ligrec_1st_5w_dict, sim_lig_rec_1st_5w)

In [638]:
sim_lig_rec_1st_9w = sim_lig_rec(rec_1st_9w, lig_1st_9w, PairsLigRec_mm_S, repeats = 10000)

In [687]:
quant_sim_lig_rec_1st_9w = sim_lig_reg_get_p(quant_ligrec_1st_9w_dict, sim_lig_rec_1st_9w)

### Get network - anagen (5w) (top 15 connections)

In [59]:
from operator import itemgetter

In [60]:
quant_ligrec_1st_5w_dict_s = sorted(quant_ligrec_1st_5w_dict.items(), key=itemgetter(1))

In [61]:
gr_sel = [gr[0] for gr in quant_ligrec_1st_5w_dict_s[-15:]]

In [62]:
quant_ligrec_1st_5w_dict_s[-15:]

[('EPI - VASC', 101),
 ('VASC - VASC', 105),
 ('MEL - ANA', 109),
 ('VASC - FIB', 120),
 ('ANA - FIB', 122),
 ('ANA - VASC', 127),
 ('VASC - ANA', 133),
 ('ANA - EPI', 133),
 ('FIB - MEL', 138),
 ('FIB - EPI', 139),
 ('EPI - ANA', 161),
 ('FIB - VASC', 169),
 ('FIB - FIB', 189),
 ('ANA - ANA', 208),
 ('FIB - ANA', 234)]

In [63]:
quant_ligrec_1st_5w_df_sel = pd.DataFrame(0, index = quant_ligrec_1st_5w_df.index, 
                                          columns = quant_ligrec_1st_5w_df.columns)

for gr in gr_sel:
    quant_ligrec_1st_5w_df_sel.loc[gr.split(' - ')[0],gr.split(' - ')[1]] = quant_ligrec_1st_5w_df.loc[gr.split(' - ')[0],gr.split(' - ')[1]]

In [64]:
gr_sel = gr_1st_5w
data = quant_ligrec_1st_5w_df_sel.loc[gr_sel, gr_sel]
dict_lig, dict_rec = lig_1st_5w, rec_1st_5w
cmap = cmap_1st

G = pgv.AGraph(strict = False, 
               dpi = 300, size = 15,
               splines=False)

for n in data.index:
    G.add_node(n, 
               fillcolor = cmap[n], 
               style = 'filled', 
               shape = 'circle', 
               label = '', 
               width = (len(dict_lig[n]) + len(dict_rec[n])) * 0.001)
    
for e1,e2 in itertools.product(list(data.index), list(data.index)):
    if data.loc[e1,e2] > 0:
        G.add_edge(e1, e2, 
                   color = cmap[e1], 
                   weight = data.loc[e1,e2], 
                   penwidth = data.loc[e1,e2] * 0.025,
                   dir = 'forward', arrowhead = None)

G.draw('%s/v1.5_SupplementaryFigure8_RL_graph_1st_5w.pdf' % path_figures, prog = 'neato')

/Users/simjoo/anaconda/lib/python3.6/site-packages/pygraphviz-1.4rc1-py3.6-macosx-10.7-x86_64.egg/pygraphviz/agraph.py:1334: RuntimeWarning: Warning: Arrow type "None" unknown - ignoring

  warnings.warn(b"".join(errors).decode(self.encoding), RuntimeWarning)


In [680]:
gr_sel = [gr[0] for gr in quant_ligrec_1st_5w_dict_s[-15:]]

In [689]:
for gr in gr_sel: print(quant_sim_lig_rec_1st_5w.loc[gr.split(' - ')[0],gr.split(' - ')[1]])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [690]:
gr_sel = [gr[0] for gr in quant_ligrec_1st_9w_dict_s[-15:]]

In [691]:
for gr in gr_sel: print(quant_sim_lig_rec_1st_9w.loc[gr.split(' - ')[0],gr.split(' - ')[1]])

0.0
0.0
0.0
0.0
0.0
0.0
0.1615
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


### Get network - telogen (9w)

In [65]:
quant_ligrec_1st_9w_dict_s = sorted(quant_ligrec_1st_9w_dict.items(), key=itemgetter(1))

In [66]:
gr_sel = [gr[0] for gr in quant_ligrec_1st_9w_dict_s[-15:]]

In [67]:
quant_ligrec_1st_9w_dict_s[-15:]

[('VASC - IMM', 50),
 ('IMM - EPI', 52),
 ('IMM - FIB', 63),
 ('IMM - VASC', 66),
 ('VASC - EPI', 77),
 ('IMM - IMM', 79),
 ('FIB - IMM', 82),
 ('EPI - EPI', 98),
 ('EPI - FIB', 98),
 ('EPI - VASC', 101),
 ('VASC - VASC', 105),
 ('VASC - FIB', 120),
 ('FIB - EPI', 139),
 ('FIB - VASC', 169),
 ('FIB - FIB', 189)]

In [68]:
quant_ligrec_1st_9w_df_sel = pd.DataFrame(0, index = quant_ligrec_1st_9w_df.index, 
                                          columns = quant_ligrec_1st_9w_df.columns)

for gr in gr_sel:
    quant_ligrec_1st_9w_df_sel.loc[gr.split(' - ')[0],gr.split(' - ')[1]] = quant_ligrec_1st_9w_df.loc[gr.split(' - ')[0],gr.split(' - ')[1]]

In [69]:
gr_sel = gr_1st_9w
data = quant_ligrec_1st_9w_df_sel.loc[gr_sel, gr_sel]
dict_lig, dict_rec = lig_1st_9w, rec_1st_9w
cmap = cmap_1st

G = pgv.AGraph(strict = False, 
               dpi = 300, size = 15,
               splines=False)

for n in data.index:
    G.add_node(n, 
               fillcolor = cmap[n], 
               style = 'filled', 
               shape = 'circle', 
               label = '', 
               width = (len(dict_lig[n]) + len(dict_rec[n])) * 0.001)
        
for e1,e2 in itertools.product(list(data.index), list(data.index)):
    if data.loc[e1,e2] > 0:
        G.add_edge(e1, e2, 
                   color = cmap[e1], 
                   weight = data.loc[e1,e2], 
                   penwidth = data.loc[e1,e2] * 0.025,
                   dir = 'forward', arrowhead = None)

G.draw('%s/v1.5_SupplementaryFigure8_RL_graph_1st_9w.pdf' % path_figures, prog = 'neato')

/Users/simjoo/anaconda/lib/python3.6/site-packages/pygraphviz-1.4rc1-py3.6-macosx-10.7-x86_64.egg/pygraphviz/agraph.py:1334: RuntimeWarning: Warning: Arrow type "None" unknown - ignoring

  warnings.warn(b"".join(errors).decode(self.encoding), RuntimeWarning)


# 2nd level clusters

## Test for enrichment to define groups for time points

In [52]:
stage_quant_2nd = pd.DataFrame(index = return_unique(cl_c_2nd_sel), columns = ['count anagen',
                                                                           '% anagen',
                                                                           'count telogen',
                                                                           '% telogen',
                                                                           'proba - 5w',
                                                                           'proba - 9w'])

for gr in return_unique(cl_c_2nd_sel):
    c_sel = cl_c_1st_sel[cl_c_2nd_sel==gr].index
    count_tmp = Counter(cells_stage[c_sel])
    
    stage_quant_2nd.loc[gr,'count anagen'] = count_tmp['5w']
    stage_quant_2nd.loc[gr,'% anagen'] = stage_quant_2nd.loc[gr,'count anagen'] / len(c_sel)
    stage_quant_2nd.loc[gr,'count telogen'] = count_tmp['9w']
    stage_quant_2nd.loc[gr,'% telogen'] = stage_quant_2nd.loc[gr,'count telogen'] / len(c_sel)
    stage_quant_2nd.loc[gr,['proba - 9w','proba - 5w']] = stage_get_pval_binomial_distr(len(c_sel), perc_anagen_all, stage_quant_2nd.loc[gr,'count anagen'])

In [53]:
thr = 0.001 / 51

In [54]:
thr

1.9607843137254903e-05

In [55]:
gr_2nd_5w_un = stage_quant_2nd['proba - 5w'][stage_quant_2nd['proba - 5w']<thr].index

In [56]:
len(gr_2nd_5w_un)

25

In [57]:
gr_2nd_9w_un = stage_quant_2nd['proba - 9w'][stage_quant_2nd['proba - 9w']<thr].index

In [58]:
len(gr_2nd_9w_un)

5

In [59]:
gr_2nd_5w = [gr for gr in set(cl_c_2nd_sel) if gr not in gr_2nd_9w_un]

In [60]:
len(gr_2nd_5w)

46

In [61]:
gr_2nd_9w = [gr for gr in set(cl_c_2nd_sel) if gr not in gr_2nd_5w_un]

In [62]:
len(gr_2nd_9w)

26

### Get receptor and ligands

In [63]:
thr = -np.log10(0.001)

In [64]:
lig_2nd = {}

for gr in return_unique(cl_c_2nd_sel):
    lig_tmp = MWU_lig[gr][MWU_lig[gr]>thr].index
    lig_2nd[gr] = lig_tmp

In [65]:
for k in lig_2nd.keys():
    print(k, len(lig_2nd[k]))

EPI - 0 21
EPI - 1 38
EPI - 2 21
EPI - 3 36
EPI - 4 29
EPI - 5 11
EPI - 6 9
EPI - 7 25
EPI - 8 16
ANA - 1 17
ANA - 21 17
ANA - 0 13
ANA - 2 60
ANA - 3 53
ANA - 20 20
ANA - 16 32
ANA - 13 17
ANA - 15 34
ANA - 12 10
ANA - 17 35
ANA - 5 20
ANA - 4 12
ANA - 14 9
ANA - 6 11
ANA - 7 8
ANA - 10 7
ANA - 11 6
ANA - 18 20
ANA - 8 19
FIB - 6 101
FIB - 5 94
FIB - 4 66
FIB - 3 78
FIB - 1 24
FIB - 2 62
FIB - 0 51
IMM - 3 15
IMM - 4 41
IMM - 1 36
IMM - 5 28
IMM - 2 25
IMM - 0 16
VASC - 3 37
VASC - 2 50
VASC - 0 41
VASC - 1 34
MEL - 1 23
MEL - 2 50
MEL - 0 21
MISC - 0 0
MISC - 1 14


In [66]:
lig_2nd_5w = {k:lig_2nd[k] for k in gr_2nd_5w}
lig_2nd_9w = {k:lig_2nd[k] for k in gr_2nd_9w}

In [67]:
rec_2nd = {}

for gr in return_unique(cl_c_2nd_sel):
    rec_tmp = MWU_rec[gr][MWU_rec[gr]>thr].index
    rec_2nd[gr] = rec_tmp

In [68]:
for k in rec_2nd.keys():
    print(k, len(rec_2nd[k]))

EPI - 0 23
EPI - 1 30
EPI - 2 23
EPI - 3 50
EPI - 4 19
EPI - 5 6
EPI - 6 7
EPI - 7 18
EPI - 8 11
ANA - 1 9
ANA - 21 17
ANA - 0 16
ANA - 2 44
ANA - 3 37
ANA - 20 23
ANA - 16 32
ANA - 13 18
ANA - 15 31
ANA - 12 5
ANA - 17 38
ANA - 5 29
ANA - 4 9
ANA - 14 10
ANA - 6 15
ANA - 7 11
ANA - 10 18
ANA - 11 12
ANA - 18 36
ANA - 8 48
FIB - 6 41
FIB - 5 37
FIB - 4 19
FIB - 3 31
FIB - 1 13
FIB - 2 37
FIB - 0 39
IMM - 3 25
IMM - 4 34
IMM - 1 38
IMM - 5 29
IMM - 2 33
IMM - 0 22
VASC - 3 54
VASC - 2 65
VASC - 0 28
VASC - 1 46
MEL - 1 25
MEL - 2 55
MEL - 0 14
MISC - 0 0
MISC - 1 12


In [69]:
rec_2nd_5w = {k:rec_2nd[k] for k in gr_2nd_5w}
rec_2nd_9w = {k:rec_2nd[k] for k in gr_2nd_9w}

In [70]:
for k in rec_2nd.keys():
    print(k, len(rec_2nd[k]) + len(lig_2nd[k]))

EPI - 0 44
EPI - 1 68
EPI - 2 44
EPI - 3 86
EPI - 4 48
EPI - 5 17
EPI - 6 16
EPI - 7 43
EPI - 8 27
ANA - 1 26
ANA - 21 34
ANA - 0 29
ANA - 2 104
ANA - 3 90
ANA - 20 43
ANA - 16 64
ANA - 13 35
ANA - 15 65
ANA - 12 15
ANA - 17 73
ANA - 5 49
ANA - 4 21
ANA - 14 19
ANA - 6 26
ANA - 7 19
ANA - 10 25
ANA - 11 18
ANA - 18 56
ANA - 8 67
FIB - 6 142
FIB - 5 131
FIB - 4 85
FIB - 3 109
FIB - 1 37
FIB - 2 99
FIB - 0 90
IMM - 3 40
IMM - 4 75
IMM - 1 74
IMM - 5 57
IMM - 2 58
IMM - 0 38
VASC - 3 91
VASC - 2 115
VASC - 0 69
VASC - 1 80
MEL - 1 48
MEL - 2 105
MEL - 0 35
MISC - 0 0
MISC - 1 26


In [71]:
for k in rec_2nd.keys():
    print(k, np.sqrt((len(rec_2nd[k]) + len(lig_2nd[k])) / np.pi))

EPI - 0 3.742410318509555
EPI - 1 4.652426491681278
EPI - 2 3.742410318509555
EPI - 3 5.232078956954491
EPI - 4 3.9088200952233594
EPI - 5 2.326213245840639
EPI - 6 2.256758334191025
EPI - 7 3.6996385101659595
EPI - 8 2.9316150714175193
ANA - 1 2.876813695875796
ANA - 21 3.2897623212397704
ANA - 0 3.0382538898732494
ANA - 2 5.753627391751592
ANA - 3 5.352372348458314
ANA - 20 3.6996385101659595
ANA - 16 4.51351666838205
ANA - 13 3.3377905890622728
ANA - 15 4.54864184146723
ANA - 12 2.1850968611841584
ANA - 17 4.820437914901168
ANA - 5 3.949327084834294
ANA - 4 2.5854414729132054
ANA - 14 2.4592453796829674
ANA - 6 2.876813695875796
ANA - 7 2.4592453796829674
ANA - 10 2.8209479177387813
ANA - 11 2.393653682408596
ANA - 18 4.222008245644752
ANA - 8 4.61809077155419
FIB - 6 6.723094811029982
FIB - 5 6.457444935117649
FIB - 4 5.201570947860099
FIB - 3 5.890312181373172
FIB - 1 3.431831258788849
FIB - 2 5.613615477764332
FIB - 0 5.352372348458314
IMM - 3 3.5682482323055424
IMM - 4 4.8860251

### Get receptor-ligand pairs

In [182]:
ligrec_2nd_5w_df, ligrec_2nd_5w_dict, quant_ligrec_2nd_5w_df, quant_ligrec_2nd_5w_dict = quantify_LigRec(lig_2nd_5w, 
                                                                                                         rec_2nd_5w,
                                                                                                         PairsLigRec_mm_S)

In [183]:
ligrec_2nd_9w_df, ligrec_2nd_9w_dict, quant_ligrec_2nd_9w_df, quant_ligrec_2nd_9w_dict = quantify_LigRec(lig_2nd_9w, 
                                                                                                         rec_2nd_9w,
                                                                                                         PairsLigRec_mm_S)

### Get network - anagen (5w)

In [93]:
gr_epidermis = [gr for gr in quant_ligrec_2nd_5w_df.index if gr.split(' - ')[0] in ['EPI','ANA']]
gr_dermis = [gr for gr in quant_ligrec_2nd_5w_df.index if gr.split(' - ')[0] in ['FIB','IMM','VASC','MEL','MISC']]

quant_ligrec_2nd_5w_df_sel = quant_ligrec_2nd_5w_df.copy()
quant_ligrec_2nd_5w_df_sel = quant_ligrec_2nd_5w_df_sel.loc[gr_epidermis + gr_dermis, gr_epidermis + gr_dermis]
quant_ligrec_2nd_5w_df_sel.loc[gr_epidermis, gr_epidermis] = 0
quant_ligrec_2nd_5w_df_sel.loc[gr_dermis, gr_dermis] = 0

In [94]:
quant_ligrec_2nd_5w_dict_sel = {}

for ix in quant_ligrec_2nd_5w_df_sel.index:
    for col in quant_ligrec_2nd_5w_df_sel.columns:
        quant_ligrec_2nd_5w_dict_sel['%s = %s' % (ix, col)] = quant_ligrec_2nd_5w_df_sel.loc[ix, col]

In [95]:
quant_ligrec_2nd_5w_dict_sel_s = sorted(quant_ligrec_2nd_5w_dict_sel.items(), key=itemgetter(1))

In [96]:
gr_sel = [gr[0] for gr in quant_ligrec_2nd_5w_dict_sel_s[-15:]]

In [97]:
quant_ligrec_2nd_5w_dict_sel_s[-15:]

[('FIB - 3 = ANA - 17', 39),
 ('FIB - 2 = ANA - 17', 39),
 ('ANA - 2 = MEL - 2', 40),
 ('FIB - 2 = ANA - 5', 40),
 ('ANA - 3 = VASC - 2', 41),
 ('FIB - 3 = ANA - 3', 43),
 ('ANA - 2 = FIB - 2', 45),
 ('FIB - 3 = ANA - 5', 45),
 ('FIB - 6 = ANA - 3', 49),
 ('ANA - 2 = VASC - 2', 50),
 ('FIB - 6 = ANA - 17', 51),
 ('FIB - 2 = ANA - 2', 55),
 ('FIB - 6 = ANA - 5', 56),
 ('FIB - 3 = ANA - 2', 59),
 ('FIB - 6 = ANA - 2', 62)]

In [98]:
quant_ligrec_2nd_5w_df_sel_ = pd.DataFrame(0, index = quant_ligrec_2nd_5w_df_sel.index, 
                                           columns = quant_ligrec_2nd_5w_df_sel.columns)

for gr in gr_sel:
    quant_ligrec_2nd_5w_df_sel_.loc[gr.split(' = ')[0],gr.split(' = ')[1]] = quant_ligrec_2nd_5w_df_sel.loc[gr.split(' = ')[0],gr.split(' = ')[1]]

In [99]:
data =  quant_ligrec_2nd_5w_df_sel_
dict_lig, dict_rec = lig_2nd_5w, rec_2nd_5w
cmap = cmap_1st

G = pgv.AGraph(strict = False, 
               dpi = 300, size = 15,
               splines=False)

for n in data.index:
    G.add_node(n, 
               fillcolor = cmap[n.split(' - ')[0]], 
               style = 'filled', 
               shape = 'circle', 
               label = n, 
               width = (len(dict_lig[n]) + len(dict_rec[n])) * 0.005)
    
for e1,e2 in itertools.product(list(data.index), list(data.index)):
    if data.loc[e1,e2] > 0:
        G.add_edge(e1, e2, 
                   color = cmap[e1.split(' - ')[0]], 
                   weight = data.loc[e1,e2], 
                   penwidth = data.loc[e1,e2] * 0.25,
                   dir = 'forward', arrowhead = None)

G.draw('%s/v1.5_SupplementaryFigure8_RL_graph_2nd_5w_lab.pdf' % path_figures, prog = 'neato')

/Users/simjoo/anaconda/lib/python3.6/site-packages/pygraphviz-1.4rc1-py3.6-macosx-10.7-x86_64.egg/pygraphviz/agraph.py:1334: RuntimeWarning: Warning: Arrow type "None" unknown - ignoring

  warnings.warn(b"".join(errors).decode(self.encoding), RuntimeWarning)


In [101]:
data =  quant_ligrec_2nd_5w_df_sel_
dict_lig, dict_rec = lig_2nd_5w, rec_2nd_5w
cmap = cmap_1st

G = pgv.AGraph(strict = False, 
               dpi = 300, size = 15,
               splines=False)

for n in data.index:
    G.add_node(n, 
               fillcolor = cmap[n.split(' - ')[0]], 
               style = 'filled', 
               shape = 'circle', 
               label = '', 
               width = (len(dict_lig[n]) + len(dict_rec[n])) * 0.005)
    
for e1,e2 in itertools.product(list(data.index), list(data.index)):
    if data.loc[e1,e2] > 0:
        G.add_edge(e1, e2, 
                   color = cmap[e1.split(' - ')[0]], 
                   weight = data.loc[e1,e2], 
                   penwidth = data.loc[e1,e2] * 0.25,
                   dir = 'forward', arrowhead = None)

G.draw('%s/v1.5_SupplementaryFigure8_RL_graph_2nd_5w.pdf' % path_figures, prog = 'neato')

/Users/simjoo/anaconda/lib/python3.6/site-packages/pygraphviz-1.4rc1-py3.6-macosx-10.7-x86_64.egg/pygraphviz/agraph.py:1334: RuntimeWarning: Warning: Arrow type "None" unknown - ignoring

  warnings.warn(b"".join(errors).decode(self.encoding), RuntimeWarning)


### Get network - anagen (9w)

In [102]:
gr_epidermis = [gr for gr in quant_ligrec_2nd_9w_df.index if gr.split(' - ')[0] in ['EPI','ANA']]
gr_dermis = [gr for gr in quant_ligrec_2nd_9w_df.index if gr.split(' - ')[0] in ['FIB','IMM','VASC','MEL','MISC']]

quant_ligrec_2nd_9w_df_sel = quant_ligrec_2nd_9w_df.copy()
quant_ligrec_2nd_9w_df_sel = quant_ligrec_2nd_9w_df_sel.loc[gr_epidermis + gr_dermis, gr_epidermis + gr_dermis]
quant_ligrec_2nd_9w_df_sel.loc[gr_epidermis, gr_epidermis] = 0
quant_ligrec_2nd_9w_df_sel.loc[gr_dermis, gr_dermis] = 0

In [103]:
quant_ligrec_2nd_9w_dict_sel = {}

for ix in quant_ligrec_2nd_9w_df_sel.index:
    for col in quant_ligrec_2nd_9w_df_sel.columns:
        quant_ligrec_2nd_9w_dict_sel['%s = %s' % (ix, col)] = quant_ligrec_2nd_9w_df_sel.loc[ix, col]

In [104]:
quant_ligrec_2nd_9w_dict_sel_s = sorted(quant_ligrec_2nd_9w_dict_sel.items(), key=itemgetter(1))

In [105]:
gr_sel = [gr[0] for gr in quant_ligrec_2nd_9w_dict_sel_s[-15:]]

In [106]:
quant_ligrec_2nd_9w_dict_sel_s[-15:]

[('EPI - 1 = VASC - 1', 17),
 ('VASC - 0 = EPI - 1', 17),
 ('FIB - 5 = EPI - 3', 17),
 ('EPI - 1 = VASC - 3', 18),
 ('FIB - 0 = EPI - 1', 19),
 ('EPI - 3 = VASC - 2', 20),
 ('FIB - 0 = EPI - 3', 20),
 ('FIB - 5 = EPI - 4', 22),
 ('FIB - 6 = EPI - 4', 22),
 ('FIB - 6 = EPI - 3', 22),
 ('EPI - 1 = VASC - 2', 27),
 ('FIB - 4 = EPI - 1', 27),
 ('EPI - 4 = VASC - 2', 28),
 ('FIB - 5 = EPI - 1', 37),
 ('FIB - 6 = EPI - 1', 38)]

In [107]:
quant_ligrec_2nd_9w_df_sel_ = pd.DataFrame(0, index = quant_ligrec_2nd_9w_df_sel.index, 
                                           columns = quant_ligrec_2nd_9w_df_sel.columns)

for gr in gr_sel:
    quant_ligrec_2nd_9w_df_sel_.loc[gr.split(' = ')[0],gr.split(' = ')[1]] = quant_ligrec_2nd_9w_df_sel.loc[gr.split(' = ')[0],gr.split(' = ')[1]]

In [110]:
data =  quant_ligrec_2nd_9w_df_sel_
dict_lig, dict_rec = lig_2nd_9w, rec_2nd_9w
cmap = cmap_1st

G = pgv.AGraph(strict = False, 
               dpi = 300, size = 15,
               splines=False)

for n in data.index:
    G.add_node(n, 
               fillcolor = cmap[n.split(' - ')[0]], 
               style = 'filled', 
               shape = 'circle', 
               label = n, 
               width = (len(dict_lig[n]) + len(dict_rec[n])) * 0.005)
    
for e1,e2 in itertools.product(list(data.index), list(data.index)):
    if data.loc[e1,e2] > 0:
        G.add_edge(e1, e2, 
                   color = cmap[e1.split(' - ')[0]], 
                   weight = data.loc[e1,e2], 
                   penwidth = data.loc[e1,e2] * 0.25,
                   dir = 'forward', arrowhead = None)

G.draw('%s/v1.5_SupplementaryFigure8_RL_graph_2nd_9w_alb.pdf' % path_figures, prog = 'neato')

/Users/simjoo/anaconda/lib/python3.6/site-packages/pygraphviz-1.4rc1-py3.6-macosx-10.7-x86_64.egg/pygraphviz/agraph.py:1334: RuntimeWarning: Warning: Arrow type "None" unknown - ignoring

  warnings.warn(b"".join(errors).decode(self.encoding), RuntimeWarning)


In [111]:
data =  quant_ligrec_2nd_9w_df_sel_
dict_lig, dict_rec = lig_2nd_9w, rec_2nd_9w
cmap = cmap_1st

G = pgv.AGraph(strict = False, 
               dpi = 300, size = 15,
               splines=False)

for n in data.index:
    G.add_node(n, 
               fillcolor = cmap[n.split(' - ')[0]], 
               style = 'filled', 
               shape = 'circle', 
               label = '', 
               width = (len(dict_lig[n]) + len(dict_rec[n])) * 0.005)
    
for e1,e2 in itertools.product(list(data.index), list(data.index)):
    if data.loc[e1,e2] > 0:
        G.add_edge(e1, e2, 
                   color = cmap[e1.split(' - ')[0]], 
                   weight = data.loc[e1,e2], 
                   penwidth = data.loc[e1,e2] * 0.25,
                   dir = 'forward', arrowhead = None)

G.draw('%s/v1.5_SupplementaryFigure8_RL_graph_2nd_9w.pdf' % path_figures, prog = 'neato')

/Users/simjoo/anaconda/lib/python3.6/site-packages/pygraphviz-1.4rc1-py3.6-macosx-10.7-x86_64.egg/pygraphviz/agraph.py:1334: RuntimeWarning: Warning: Arrow type "None" unknown - ignoring

  warnings.warn(b"".join(errors).decode(self.encoding), RuntimeWarning)


# Get sets of interactions

In [223]:
quant_ligrec_2nd_5w_dict_sel_s[-15:]

[('FIB - 2 = ANA - 17', 39),
 ('FIB - 3 = ANA - 17', 39),
 ('ANA - 2 = MEL - 2', 40),
 ('FIB - 2 = ANA - 5', 40),
 ('ANA - 3 = VASC - 2', 41),
 ('FIB - 3 = ANA - 3', 43),
 ('ANA - 2 = FIB - 2', 45),
 ('FIB - 3 = ANA - 5', 45),
 ('FIB - 6 = ANA - 3', 49),
 ('ANA - 2 = VASC - 2', 50),
 ('FIB - 6 = ANA - 17', 51),
 ('FIB - 2 = ANA - 2', 55),
 ('FIB - 6 = ANA - 5', 56),
 ('FIB - 3 = ANA - 2', 59),
 ('FIB - 6 = ANA - 2', 62)]

In [215]:
list_tmp = []

for k in ['FIB - 6 - EPI - 1',
          'FIB - 5 - EPI - 1',
          'FIB - 4 - EPI - 1',
          'FIB - 0 - EPI - 1']:
    
    list_tmp += ligrec_2nd_9w_dict[k]

In [220]:
list_tmp = []

for k in ['EPI - 4 - VASC - 2',
            'EPI - 1 - VASC - 2',
            'EPI - 3 - VASC - 2',
            'EPI - 1 - VASC - 3',
            'EPI - 1 - VASC - 1']:
    
    list_tmp += ligrec_2nd_9w_dict[k]

In [224]:
list_tmp = []

for k in ['FIB - 6 - ANA - 2',
          'FIB - 3 - ANA - 2',
          'FIB - 2 - ANA - 2',
          'FIB - 6 - ANA - 3',
          'FIB - 3 - ANA - 3']:
    
    list_tmp += ligrec_2nd_5w_dict[k]

In [227]:
list_tmp = []

for k in ['FIB - 6 - ANA - 5',
          'FIB - 6 - ANA - 17',
          'FIB - 3 - ANA - 5',
          'FIB - 2 - ANA - 5',
          'FIB - 3 - ANA - 17',
          'FIB - 2 - ANA - 17']:
    
    list_tmp += ligrec_2nd_5w_dict[k]

In [230]:
list_tmp = []

for k in ['ANA - 2 - MEL - 2']:
    
    list_tmp += ligrec_2nd_5w_dict[k]

In [233]:
list_tmp = []

for k in ['ANA - 2 - FIB - 2']:
    
    list_tmp += ligrec_2nd_5w_dict[k]

In [236]:
list_tmp = []

for k in ['ANA - 2 - VASC - 2',
          'ANA - 3 - VASC - 2']:
    
    list_tmp += ligrec_2nd_5w_dict[k]

# Interaction potential of ORS cells with skin fibroblasts

In [183]:
lig_sFIB = []

for k in ['FIB - 6','FIB - 5','FIB - 4','FIB - 3']:
    lig_sFIB += list(lig_2nd[k])

lig_sFIB = list(set(lig_sFIB))

In [147]:
len(lig_sFIB)

127

In [148]:
rec_sFIB = []

for k in ['FIB - 6','FIB - 5','FIB - 4','FIB - 3']:
    rec_sFIB += list(rec_2nd[k])

rec_sFIB = list(set(rec_sFIB))

In [150]:
len(rec_sFIB)

59

In [153]:
lig_2nd_ORS = {}

lig_2nd_ORS['sFIB'] = lig_sFIB

for k in ['ANA - 1','ANA - 21','ANA - 0', 'ANA - 2', 'ANA - 3', 'ANA - 20']:
    lig_2nd_ORS[k] = lig_2nd[k]

In [154]:
rec_2nd_ORS = {}

rec_2nd_ORS['sFIB'] = rec_sFIB

for k in ['ANA - 1','ANA - 21','ANA - 0', 'ANA - 2', 'ANA - 3', 'ANA - 20']:
    rec_2nd_ORS[k] = rec_2nd[k]   

In [155]:
ligrec_2nd_ORS_df, ligrec_2nd_ORS_dict, quant_ligrec_2nd_ORS_df, quant_ligrec_2nd_ORS_dict = quantify_LigRec(lig_2nd_ORS, 
                                                                                                         rec_2nd_ORS,
                                                                                                         PairsLigRec_mm_S)

In [158]:
quant_ligrec_2nd_ORS_df_ = pd.DataFrame(0, index = quant_ligrec_2nd_ORS_df.index, columns = quant_ligrec_2nd_ORS_df.columns)

for c in ['ANA - 1','ANA - 21','ANA - 0', 'ANA - 2', 'ANA - 3', 'ANA - 20']:
    quant_ligrec_2nd_ORS_df_.loc['sFIB',c] = quant_ligrec_2nd_ORS_df.loc['sFIB',c]

In [168]:
quant_ligrec_2nd_ORS_df_

,sFIB,ANA - 1,ANA - 21,ANA - 0,ANA - 2,ANA - 3,ANA - 20
sFIB,0,8,23,21,81,61,44
ANA - 1,0,0,0,0,0,0,0
ANA - 21,0,0,0,0,0,0,0
ANA - 0,0,0,0,0,0,0,0
ANA - 2,0,0,0,0,0,0,0
ANA - 3,0,0,0,0,0,0,0
ANA - 20,0,0,0,0,0,0,0


In [161]:
data =  quant_ligrec_2nd_ORS_df_
dict_lig, dict_rec = lig_2nd_ORS, rec_2nd_ORS
cmap = cmap_2nd
cmap['sFIB'] = 'red'

G = pgv.AGraph(strict = False, 
               dpi = 300, size = 15,
               splines=False)

for n in data.index:
    G.add_node(n, 
               fillcolor = cmap[n],#cmap[n.split(' - ')[0]], 
               style = 'filled', 
               shape = 'circle', 
               label = '', 
               width = (len(dict_lig[n]) + len(dict_rec[n])) * 0.005)
    
for e1,e2 in itertools.product(list(data.index), list(data.index)):
    if data.loc[e1,e2] > 0:
        G.add_edge(e1, e2, 
                   color = cmap[e1],#cmap[e1.split(' - ')[0]], 
                   weight = data.loc[e1,e2], 
                   penwidth = data.loc[e1,e2] * 0.25,
                   dir = 'forward', arrowhead = None)

G.draw('%s/v1.5_SupplementaryFigure8_RL_graph_2nd_5w_ORS.pdf' % path_figures, prog = 'neato')

/Users/simjoo/anaconda/lib/python3.6/site-packages/pygraphviz-1.4rc1-py3.6-macosx-10.7-x86_64.egg/pygraphviz/agraph.py:1334: RuntimeWarning: Warning: Arrow type "None" unknown - ignoring

  warnings.warn(b"".join(errors).decode(self.encoding), RuntimeWarning)


In [164]:
list_tmp = []

for k in ['sFIB - ANA - 2','sFIB - ANA - 3']:
    
    list_tmp += ligrec_2nd_ORS_dict[k]

# Interaction potential of ORS cells with stromal cells

In [184]:
gr_stroma = [g for g in set(cl_c_2nd_sel) if g.split(' - ')[0] not in ['ANA','EPI']]

In [186]:
lig_stroma = []

for k in gr_stroma:
    lig_stroma += list(lig_2nd[k])

lig_stroma = list(set(lig_stroma))

In [187]:
len(lig_stroma)

261

In [188]:
rec_stroma = []

for k in gr_stroma:
    rec_stroma += list(rec_2nd[k])

rec_stroma = list(set(rec_stroma))

In [189]:
len(rec_stroma)

235

In [190]:
lig_2nd_ORS = {}

lig_2nd_ORS['stroma'] = lig_stroma

for k in ['ANA - 1','ANA - 21','ANA - 0', 'ANA - 2', 'ANA - 3', 'ANA - 20']:
    lig_2nd_ORS[k] = lig_2nd[k]

In [196]:
rec_2nd_ORS = {}

rec_2nd_ORS['stroma'] = rec_stroma

for k in ['ANA - 1','ANA - 21','ANA - 0', 'ANA - 2', 'ANA - 3', 'ANA - 20']:
    rec_2nd_ORS[k] = rec_2nd[k]   

In [197]:
ligrec_2nd_ORS_df, ligrec_2nd_ORS_dict, quant_ligrec_2nd_ORS_df, quant_ligrec_2nd_ORS_dict = quantify_LigRec(lig_2nd_ORS, 
                                                                                                         rec_2nd_ORS,
                                                                                                         PairsLigRec_mm_S)

In [198]:
quant_ligrec_2nd_ORS_df_ = pd.DataFrame(0, index = quant_ligrec_2nd_ORS_df.index, columns = quant_ligrec_2nd_ORS_df.columns)

for c in ['ANA - 1','ANA - 21','ANA - 0', 'ANA - 2', 'ANA - 3', 'ANA - 20']:
    quant_ligrec_2nd_ORS_df_.loc['stroma',c] = quant_ligrec_2nd_ORS_df.loc['stroma',c]

In [199]:
quant_ligrec_2nd_ORS_df_

,stroma,ANA - 1,ANA - 21,ANA - 0,ANA - 2,ANA - 3,ANA - 20
stroma,0,19,45,37,156,115,78
ANA - 1,0,0,0,0,0,0,0
ANA - 21,0,0,0,0,0,0,0
ANA - 0,0,0,0,0,0,0,0
ANA - 2,0,0,0,0,0,0,0
ANA - 3,0,0,0,0,0,0,0
ANA - 20,0,0,0,0,0,0,0


In [204]:
data =  quant_ligrec_2nd_ORS_df_
dict_lig, dict_rec = lig_2nd_ORS, rec_2nd_ORS
cmap = cmap_2nd
cmap['stroma'] = 'red'

G = pgv.AGraph(strict = False, 
               dpi = 300, size = 15,
               splines=False)

for n in data.index:
    G.add_node(n, 
               fillcolor = cmap[n],#cmap[n.split(' - ')[0]], 
               style = 'filled', 
               shape = 'circle', 
               label = '', 
               width = 0.25)
    
for e1,e2 in itertools.product(list(data.index), list(data.index)):
    if data.loc[e1,e2] > 0:
        G.add_edge(e1, e2, 
                   color = cmap[e1],#cmap[e1.split(' - ')[0]], 
                   weight = data.loc[e1,e2], 
                   penwidth = data.loc[e1,e2] * 0.25,
                   dir = 'forward', arrowhead = None)

G.draw('%s/v1.5_SupplementaryFigure8_RL_graph_2nd_5w_ORS.pdf' % path_figures, prog = 'neato')

/Users/simjoo/anaconda/lib/python3.6/site-packages/pygraphviz-1.4rc1-py3.6-macosx-10.7-x86_64.egg/pygraphviz/agraph.py:1334: RuntimeWarning: Warning: Arrow type "None" unknown - ignoring

  warnings.warn(b"".join(errors).decode(self.encoding), RuntimeWarning)


# Interaction potential of ORS cells with main stromal compartments

In [216]:
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout

In [234]:
from IPython.display import Image, SVG

In [214]:
ligrec_2nd_ORS_df, ligrec_2nd_ORS_dict, quant_ligrec_2nd_ORS_df, quant_ligrec_2nd_ORS_dict = quantify_LigRec(lig_1st_5w, 
                                                                                                         rec_2nd_ORS,
                                                                                                         PairsLigRec_mm_S)

In [215]:
quant_ligrec_2nd_ORS_df

,stroma,ANA - 1,ANA - 21,ANA - 0,ANA - 2,ANA - 3,ANA - 20
IMM,191,5,11,14,36,25,14
MISC,37,1,2,1,11,5,4
MEL,137,7,20,10,47,34,23
VASC,213,7,16,16,63,43,29
ANA,241,12,33,30,93,69,43
EPI,204,9,21,20,64,40,34
FIB,358,13,31,27,113,77,57


In [285]:
stroma_ORS_links = []

for x1 in ['FIB','IMM','VASC','MEL','MISC']:
    for x2 in ['ANA - 1','ANA - 21','ANA - 0','ANA - 2','ANA - 3','ANA - 20']:
        stroma_ORS_links += [{'source':x1,
                              'target':x2,
                              'value':quant_ligrec_2nd_ORS_df.loc[x1,x2],
                              'color':cmap_2nd[x2]}]

In [286]:
stroma_ORS_order = [['FIB','IMM','VASC','MEL','MISC'],['ANA - 1','ANA - 21','ANA - 0','ANA - 2','ANA - 3','ANA - 20']]

In [287]:
layout = Layout(width="500", height="500")
w = SankeyWidget(links = stroma_ORS_links,
                 margins=dict(top=0, bottom=0, left=100, right=100), 
                 layout = layout,
                 order = stroma_ORS_order,
                )

In [288]:
w

SankeyWidget(layout=Layout(height='500', width='500'), links=[{'source': 'FIB', 'target': 'ANA - 1', 'value': …

In [289]:
w.save_svg('%s/sankay_stroma_vs_ORS_v2.svg' % path_figures)